In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

from buttermilk import BM
from promptflow.tracing import trace, start_trace
from rich import print as rprint

import os

from hydra import initialize, compose
from omegaconf import OmegaConf

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+data=drag", "+step=ordinary", "+save=bq", "job=score","source=judger"])

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)

# Print config
rprint(OmegaConf.to_container(bm.cfg, resolve=True))



/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-11-04 21:09:38 J5HW6L4KT6 buttermilk buttermilk.py[ 297] INFO {'message': "Logging setup for: 20241104T1109Z-naPB-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'automod', 'job': 'score', 'task_id': '20241104T1109Z-naPB-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/automod/score/20241104T1109Z-naPB-J5HW6L4KT6-suzor", 'project': 'automod', 'job': 'score', 'run_id': '20241104T1109Z-naPB-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/automod/score/20241104T1109Z-naPB-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
2024-11-04 21:09:38 J5HW6L4KT6 buttermilk buttermilk.py[ 305] DEBUG Buttermilk version is: 0.2.0
Prompt flow service has started...


{
    'name': 'automod',
    'job': 'score',
    'source': 'judger',
    'secret_provider': 'azure',
    'logger': 'gcp',
    'tracing': True,
    'verbose': True,
    'models_secret': 'models',
    'save_dest': 'gcp',
    'save_dir': None,
    'gcp': {'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'azure': {'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 'rg-suzor_ai'},
    'aws': {'secret_name': 'secret', 'region': 'us-east-1'},
    'step': [
        {
            'name': 'judger',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc_judge',
                'template': 'judge',
                'criteria': 'criteria_ordinary',
                'formatting': 'json_rules',
                'model': ['haiku', 'llama31_70b', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': None
        }
    ],
    'run': {'platform': 'local'},
    'data': {
        'drag': {
            'name': 'drag queens - alt text',
            'type': 'file',
            'uri': 'gs://dmrc-platforms/data/drag_train.jsonl',
            'columns': {'record_id': 'id', 'content': 'alt_text', 'groundtruth': 'expected'}
        }
    },
    'save': {'type': 'bq', 'destination': 'dmrc-analysis.toxicity.flow', 'schema': 'flow.json'}
}

In [3]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[0], data=cfg.data, save=cfg.save, source=cfg.source)


I0000 00:00:1730718580.277649 17651423 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [ ]:
await orchestrator.run_tasks()
